# Assignment 6
## Project 1
The first project you can choose is a continuation of the InterPROscan dataset you worked on for Assignment 5. You now know the structure and type of data that is involved. What I would ike you to do is to develop scikit-learn machine learning models to predict the function of the proteins in the dataset. I'm not going to mandate any particular model, but you should use your knowledge from DS3 to determine good candidate methods and try a couple, and report on yourGiven the size of the data, you should train your machine models in parallel! Dask is an obvious candidate for this, but you can also use any of the other frameworks we discussed in the course.

The "function" of the protein which is the "class" your model should predict is defined as the InterPRO number which covers:

1. more than 90% of the protein's sequence
2. Covers the largest length of the sequence

The "features" that you use to predict that class are the other, smaller, InterPRO annotations. Take care to remove "noise" from the dataset, ie. lines from the TSV file which don't contain InterPRO numbers, proteins which don't have a large feature (according to the criteria above) etc. Be sure to note in your report how you processed the data and why you made the choices you did!

Your script should, given one or more of these TSV files with InterPROscan annotations, produce models (Pickle them to save them!) which predict protein function, and a file of training-data on which you trained them.


In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


## Data preperation

In [2]:
#path = "/data/dataprocessing/interproscan/all_bacilli.tsv"
#path = "all_bacilli.tsv"
path = "bacilli_sample_1000.tsv"
df = pd.read_csv(path, sep = "\t", names = ["Protein_acc", "Sequence_MD5", "Seq_lenght", "Analysis", "Signature_acc", 
                                            "Signature_description", "Start", "Stop", "Score", "Status", 
                                           "Data", "Interpro_acc", "Interpro_descr", "GO_anno", "Pathway_anno"])

In [3]:
df

,Protein_acc,Sequence_MD5,Seq_lenght,Analysis,Signature_acc,Signature_description,Start,Stop,Score,Status,Data,Interpro_acc,Interpro_descr,GO_anno,Pathway_anno
0,gi|29898682|gb|AAP11954.1|,92d1264e347e149248231cb9b649388c,547,TIGRFAM,TIGR03882,cyclo_dehyd_2: bacteriocin biosynthesis cyclod...,2,131,1.6E-21,T,25-04-2022,IPR022291,"Bacteriocin biosynthesis, cyclodehydratase domain",-,-
1,gi|29898682|gb|AAP11954.1|,92d1264e347e149248231cb9b649388c,547,TIGRFAM,TIGR03604,TOMM_cyclo_SagD: thiazole/oxazole-forming pept...,161,547,0.0,T,25-04-2022,IPR027624,"Thiazole/oxazole-forming peptide maturase, Sag...",-,-
2,gi|29898682|gb|AAP11954.1|,92d1264e347e149248231cb9b649388c,547,ProSiteProfiles,PS51664,YcaO domain profile.,159,547,75.396477,T,25-04-2022,IPR003776,YcaO-like domain,-,-
3,gi|29898682|gb|AAP11954.1|,92d1264e347e149248231cb9b649388c,547,Gene3D,G3DSA:3.30.160.660,-,311,452,9.1E-46,T,25-04-2022,-,-,NaN,NaN
4,gi|29898682|gb|AAP11954.1|,92d1264e347e149248231cb9b649388c,547,Gene3D,G3DSA:3.30.40.250,-,191,274,9.1E-46,T,25-04-2022,-,-,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,gi|29898029|gb|AAP11303.1|,72f7550b6af976df04f0e1ec41053415,219,SMART,SM00028,tpr_5,35,68,0.0021,T,25-04-2022,IPR019734,Tetratricopeptide repeat,GO:0005515,MetaCyc: PWY-1061|MetaCyc: PWY-2083|MetaCyc: P...
996,gi|29898029|gb|AAP11303.1|,72f7550b6af976df04f0e1ec41053415,219,SMART,SM00028,tpr_5,171,204,2.4E-6,T,25-04-2022,IPR019734,Tetratricopeptide repeat,GO:0005515,MetaCyc: PWY-1061|MetaCyc: PWY-2083|MetaCyc: P...
997,gi|29898029|gb|AAP11303.1|,72f7550b6af976df04f0e1ec41053415,219,SMART,SM00028,tpr_5,137,170,0.089,T,25-04-2022,IPR019734,Tetratricopeptide repeat,GO:0005515,MetaCyc: PWY-1061|MetaCyc: PWY-2083|MetaCyc: P...
998,gi|29898029|gb|AAP11303.1|,72f7550b6af976df04f0e1ec41053415,219,SMART,SM00028,tpr_5,103,136,0.62,T,25-04-2022,IPR019734,Tetratricopeptide repeat,GO:0005515,MetaCyc: PWY-1061|MetaCyc: PWY-2083|MetaCyc: P...


In [4]:
df = df[df["Interpro_acc"] != "-"]

In [5]:
df

,Protein_acc,Sequence_MD5,Seq_lenght,Analysis,Signature_acc,Signature_description,Start,Stop,Score,Status,Data,Interpro_acc,Interpro_descr,GO_anno,Pathway_anno
0,gi|29898682|gb|AAP11954.1|,92d1264e347e149248231cb9b649388c,547,TIGRFAM,TIGR03882,cyclo_dehyd_2: bacteriocin biosynthesis cyclod...,2,131,1.6E-21,T,25-04-2022,IPR022291,"Bacteriocin biosynthesis, cyclodehydratase domain",-,-
1,gi|29898682|gb|AAP11954.1|,92d1264e347e149248231cb9b649388c,547,TIGRFAM,TIGR03604,TOMM_cyclo_SagD: thiazole/oxazole-forming pept...,161,547,0.0,T,25-04-2022,IPR027624,"Thiazole/oxazole-forming peptide maturase, Sag...",-,-
2,gi|29898682|gb|AAP11954.1|,92d1264e347e149248231cb9b649388c,547,ProSiteProfiles,PS51664,YcaO domain profile.,159,547,75.396477,T,25-04-2022,IPR003776,YcaO-like domain,-,-
6,gi|29898682|gb|AAP11954.1|,92d1264e347e149248231cb9b649388c,547,TIGRFAM,TIGR00702,TIGR00702: YcaO-type kinase domain,161,501,2.2E-22,T,25-04-2022,IPR003776,YcaO-like domain,-,-
8,gi|29898682|gb|AAP11954.1|,92d1264e347e149248231cb9b649388c,547,Pfam,PF02624,"YcaO cyclodehydratase, ATP-ad Mg2+-binding",161,501,8.6E-76,T,25-04-2022,IPR003776,YcaO-like domain,-,-
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,gi|29898029|gb|AAP11303.1|,72f7550b6af976df04f0e1ec41053415,219,SMART,SM00028,tpr_5,35,68,0.0021,T,25-04-2022,IPR019734,Tetratricopeptide repeat,GO:0005515,MetaCyc: PWY-1061|MetaCyc: PWY-2083|MetaCyc: P...
996,gi|29898029|gb|AAP11303.1|,72f7550b6af976df04f0e1ec41053415,219,SMART,SM00028,tpr_5,171,204,2.4E-6,T,25-04-2022,IPR019734,Tetratricopeptide repeat,GO:0005515,MetaCyc: PWY-1061|MetaCyc: PWY-2083|MetaCyc: P...
997,gi|29898029|gb|AAP11303.1|,72f7550b6af976df04f0e1ec41053415,219,SMART,SM00028,tpr_5,137,170,0.089,T,25-04-2022,IPR019734,Tetratricopeptide repeat,GO:0005515,MetaCyc: PWY-1061|MetaCyc: PWY-2083|MetaCyc: P...
998,gi|29898029|gb|AAP11303.1|,72f7550b6af976df04f0e1ec41053415,219,SMART,SM00028,tpr_5,103,136,0.62,T,25-04-2022,IPR019734,Tetratricopeptide repeat,GO:0005515,MetaCyc: PWY-1061|MetaCyc: PWY-2083|MetaCyc: P...


In [6]:
df = df[["Protein_acc", "Seq_lenght", "Start", "Stop", "Interpro_acc"]]

In [7]:
df

,Protein_acc,Seq_lenght,Start,Stop,Interpro_acc
0,gi|29898682|gb|AAP11954.1|,547,2,131,IPR022291
1,gi|29898682|gb|AAP11954.1|,547,161,547,IPR027624
2,gi|29898682|gb|AAP11954.1|,547,159,547,IPR003776
6,gi|29898682|gb|AAP11954.1|,547,161,501,IPR003776
8,gi|29898682|gb|AAP11954.1|,547,161,501,IPR003776
...,...,...,...,...,...
995,gi|29898029|gb|AAP11303.1|,219,35,68,IPR019734
996,gi|29898029|gb|AAP11303.1|,219,171,204,IPR019734
997,gi|29898029|gb|AAP11303.1|,219,137,170,IPR019734
998,gi|29898029|gb|AAP11303.1|,219,103,136,IPR019734


In [8]:
#don't use. does not work and would take very long, use apply()
#for index, row in df2.iterrows():
    #if ((df2["Stop"] - df2["Start"]) / df2["Seq_lenght"]) > 0.90:
        #print("1")
        #df2["Size"] = [1]
    #else:
        #print("0")
        #df2["size"] = [0]

In [9]:
def coverage_calc(ddf):
    """
    calculate percentage the feature covers on the total protein length
    use with .apply() method

    returns:
        0 if Interpro_acc < 0.90 (small)
        1 if Interpro_acc > 0.90 (large)
    """
    
    Size = (ddf["Stop"] - ddf["Start"]) / ddf["Seq_lenght"]
    return Size


df["Size"] = df.apply(lambda x:coverage_calc(x), axis = 1)

C:\Users\piete\AppData\Local\Temp/ipykernel_34080/3728154484.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["Size"] = df.apply(lambda x:coverage_calc(x), axis = 1)


In [10]:
df

,Protein_acc,Seq_lenght,Start,Stop,Interpro_acc,Size
0,gi|29898682|gb|AAP11954.1|,547,2,131,IPR022291,0.235832
1,gi|29898682|gb|AAP11954.1|,547,161,547,IPR027624,0.705667
2,gi|29898682|gb|AAP11954.1|,547,159,547,IPR003776,0.709324
6,gi|29898682|gb|AAP11954.1|,547,161,501,IPR003776,0.621572
8,gi|29898682|gb|AAP11954.1|,547,161,501,IPR003776,0.621572
...,...,...,...,...,...,...
995,gi|29898029|gb|AAP11303.1|,219,35,68,IPR019734,0.150685
996,gi|29898029|gb|AAP11303.1|,219,171,204,IPR019734,0.150685
997,gi|29898029|gb|AAP11303.1|,219,137,170,IPR019734,0.150685
998,gi|29898029|gb|AAP11303.1|,219,103,136,IPR019734,0.150685


In [11]:
def rem(df):
    if (df["Size"] > 0.90).any() and (df["Size"] < 0.90).any():
        return df
    
    
    
df2 = df.groupby("Protein_acc").apply(rem).reset_index(drop = True)

In [12]:
df2

,Protein_acc,Seq_lenght,Start,Stop,Interpro_acc,Size
0,gi|29893821|gb|AAP07113.1|,70,1,70,IPR036986,0.985714
1,gi|29893821|gb|AAP07113.1|,70,4,62,IPR014330,0.828571
2,gi|29893899|gb|AAP07191.1|,221,1,66,IPR042122,0.294118
3,gi|29893899|gb|AAP07191.1|,221,64,164,IPR045304,0.452489
4,gi|29893899|gb|AAP07191.1|,221,71,108,IPR001451,0.167421
...,...,...,...,...,...,...
260,gi|29899068|gb|AAP12339.1|,629,568,620,IPR044920,0.082671
261,gi|29899068|gb|AAP12339.1|,629,8,620,IPR004416,0.972973
262,gi|29899068|gb|AAP12339.1|,629,368,391,IPR020595,0.036566
263,gi|29899068|gb|AAP12339.1|,629,7,621,IPR002218,0.976153


In [24]:
df3 = df2.groupby(["Protein_acc"])["Size"].transform(max)

In [29]:
idx = df2.groupby(["Protein_acc"])["Size"].transform(max)
ddf_large = df2[idx == df2["Size"]]
ddf_small = df2[~(df2["Size"] == idx)]

In [30]:
ddf_large

,Protein_acc,Seq_lenght,Start,Stop,Interpro_acc,Size
0,gi|29893821|gb|AAP07113.1|,70,1,70,IPR036986,0.985714
6,gi|29893899|gb|AAP07191.1|,221,1,218,IPR005881,0.981900
14,gi|29893909|gb|AAP07201.1|,166,3,161,IPR022973,0.951807
21,gi|29893922|gb|AAP07214.1|,96,1,95,IPR012677,0.979167
24,gi|29893947|gb|AAP07239.1|,120,1,120,IPR000456,0.991667
29,gi|29893947|gb|AAP07239.1|,120,1,120,IPR036373,0.991667
30,gi|29894058|gb|AAP07350.1|,233,1,231,IPR039420,0.987124
43,gi|29894081|gb|AAP07372.1|,195,3,189,IPR004607,0.953846
46,gi|29894192|gb|AAP07483.1|,196,5,191,IPR003325,0.948980
54,gi|29894215|gb|AAP07506.1|,506,1,505,IPR011167,0.996047


In [31]:
ddf_small

,Protein_acc,Seq_lenght,Start,Stop,Interpro_acc,Size
1,gi|29893821|gb|AAP07113.1|,70,4,62,IPR014330,0.828571
2,gi|29893899|gb|AAP07191.1|,221,1,66,IPR042122,0.294118
3,gi|29893899|gb|AAP07191.1|,221,64,164,IPR045304,0.452489
4,gi|29893899|gb|AAP07191.1|,221,71,108,IPR001451,0.167421
5,gi|29893899|gb|AAP07191.1|,221,117,151,IPR001451,0.153846
...,...,...,...,...,...,...
260,gi|29899068|gb|AAP12339.1|,629,568,620,IPR044920,0.082671
261,gi|29899068|gb|AAP12339.1|,629,8,620,IPR004416,0.972973
262,gi|29899068|gb|AAP12339.1|,629,368,391,IPR020595,0.036566
263,gi|29899068|gb|AAP12339.1|,629,7,621,IPR002218,0.976153
